In [2]:
from choice_model import ChoiceModel
import networkx as nx


In [ ]:
model = ChoiceModel()
new_model = ChoiceModel(graph_path='your_new_graph_path') 
period = 2  

In [ ]:
# get all nodes in certain period 
graph = model.get_period(period)
new_nodes = [n for n in graph.nodes if n not in list(new_model.graph.nodes)]
new_nodes = [n for n in new_nodes if graph.nodes[n]['type'] != 'Long-term Project']
actor_ndoes = [n for n in graph.nodes if graph.nodes[n]['type'] == 'Actors']
other_nodes = [n for n in graph.nodes if graph.nodes[n]['type'] != 'Actors']
print("number of nodes: ",len(new_nodes))
print("number of actors: ", len(actor_ndoes))
print("number of other nodes: ",len(other_nodes))

In [ ]:
test_graph = new_model.graph.copy()

In [ ]:
# add each node to the test_graph

from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np

# use all history data as reference to predict
model.roll_back(9)
data_df = pd.DataFrame(columns=['node', 'profile', 'choices', 'new_options','llm_choice', 'llm_response'])
for node in tqdm(new_nodes):
    try:
        profile = model.graph.nodes[node]['properties']
        node_type = model.graph.nodes[node]['type']
        test_options = list(test_graph.nodes)
        test_graph.add_node(node, type=node_type, properties=profile, embedding=model.embed_model.embed_query(str(profile)))
        old_context,choices = model.get_old_context(profile=profile, node_type=node_type, k1=10,k2=5, top_k=5)
        choices = [ c['properties'] for c in choices]
        # find most silimar options to choices
        index = model._build_index(test_graph)
        link_options = []
        for choice in choices:
            query_embed = model.embed_model.embed_query(str(choice))
            query_embed = np.array(query_embed).reshape(1,-1)
            d, i = index['index'].search(query_embed, 1)
            similar_nodes = index['ids'][i][0]
            link_options.extend(similar_nodes.tolist())
        new_options = [test_graph.nodes[n]['properties'] for n in link_options]
        # ask llm to predict links
        llm_choice, llm_response = model.get_llm_choice(profile=profile, new_options=new_options, old_context=old_context, k1=10, k2=5, node_type=node_type,top_k=5)
        # add edges
        llm_answer = llm_choice['answer']
        for answer,id in zip(llm_answer, link_options):
             if answer=='Yes':
                print("add edge", node, id)
                test_graph.add_edge(node, id)
        data_df.loc[len(data_df)] = [node, profile, choices, new_options,llm_choice, llm_response]
        if len(data_df) % 50 == 0:
            data_df.to_csv(f'with_llm_period_{period}.csv')
            with open(f"with_llm_period_{period}.pkl", "wb") as f:
                pickle.dump(test_graph, f)
    except Exception as e:
        print(e)
        pass

data_df.to_csv(f'with_llm_period_{period}.csv')
# save the embeded graph
with open("with_llm_period_{period}.pkl", "wb") as f:
    pickle.dump(test_graph, f)


In [ ]:
PPI, CVI, DCI = new_model.evaluate(test_graph)
print("PPI: ", PPI)
print("CVI: ", CVI)
print("DCI: ", DCI)